The purpose of this notebook is to show how to **read and process** some research data using xarray library.

See xarray documentation here: (https://docs.xarray.dev/en/stable/index.html)


# general imports

In [ ]:
# We need to import some Python libraries here first.

import os
import sys
import shutil
import scipy

import numpy as np
import xarray as xr

import PIL # python image library for reading tiff files

In [ ]:
# It would be good to have interactive plots if ipympl is installed.
# See: https://matplotlib.org/ipympl/

try:
    %matplotlib inline
except:
    %matplotlib widget

# importing matplotlib for plots.
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0})
plt.rcParams['figure.constrained_layout.use'] = True

# setting up the example data path

In [ ]:
# We assume this notebook and its data is on Google Drive so that we can use Google Colab.
# Otherwise, replace example_data_path with the path the notebook and its data folder are stored on

try:
    from google.colab import drive
    drive.mount('/content/drive')
    os.chdir('/content/drive/MyDrive/RTE-24-4876/code/pySULI-main/notebooks')
except:
    pass

In [ ]:
# Assuming notebook and data folder are here.

here = os.getcwd()
example_data_path = 'example_data/file_types/image/'

os.chdir('%s/%s'%(here,example_data_path))
os.listdir()

# reading a tiff image file as a numpy array and plotting without using xarray

In [ ]:
# first we need to read image as numpy array
img = np.array(PIL.Image.open('NIST-LaB6.tiff'))

# then plot using imshow
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.imshow(img)
ax.set_aspect('equal')

This is an inefficient method for plotting; we will see how to do it with xarray instead.
# reading a tiff image file as xarray dataarray and plotting with xarray.

The general approach towards plotting using xarray is to treat the image as a 2d array; later, it will allow us to manipulate the image in a straightforward way.

In [ ]:
# first we need to read image as numpy array
img = np.array(PIL.Image.open('NIST-LaB6.tiff'))


# then we create an xarray DataArray object
da = xr.DataArray(data=img,
                  coords=[np.arange(img.shape[0]),np.arange(img.shape[1])],
                  dims=['pixel_y', 'pixel_x'])

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
da.plot.imshow(ax=ax) # this line does the plotting for us and adds colorbar and figure axis labels.
ax.set_aspect('equal')

The coloring scale `robust` is a great visual aide for examining 2d intensity plots. To utilize it, in the `imshow()` method set `robust=True` as shown below.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
da.plot.imshow(ax=ax,robust=True)
ax.set_aspect('equal')

Change the colormap with the parameter `cmap`. `vmin` and `vmax` define the range of the colormap (which you can see in the bar on the right side of the plot).

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
da.plot.imshow(ax=ax,robust=True,cmap = 'Greys', vmin=0, vmax=70)
ax.set_aspect('equal')

See below why we use xarray instead of matplotlib for making these kinds of plots:

In [ ]:
# in order to get above plot, we need to add some manual stuff

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
im = ax.imshow(img,vmin=0,vmax=70,cmap='Greys') # we learned 70 here from xarray plot

# --- xarray plotting does these for us automatically 
cb = fig.colorbar(im) 
ax.set_xlabel('pixel_x')
ax.set_ylabel('pixel_y')
# ---------------------------------------------------

ax.set_aspect('equal')

>NOTE: There are some detector defects on the image.
For example, there are defective horizontal lines on pixel_x = 102 & pixel_y = 465 in the xarray plot above.
Unfortunately these lines are invisible in the plot.

Adding x,y labels or a fancy colorbar are not the only advantages of xarray. Xarrat has plenty of
robust functionality that is not as straightforward with standard numpy and matplotlib functions.
The following examples show some of those functionalities.

# more examples with xarray

Here is where the treatment of the images as arrays allows us some cool functionality:

In [ ]:
# The features around pixel_x=1910, pixel_y=980 are coming from beam stop.
# We can easily zoom in those areas in a second plot

fig = plt.figure(figsize=(10,4))

ax = fig.add_subplot(1,2,1)
da.plot.imshow(ax=ax,robust=True,cmap = 'Greys', vmin=0, vmax=70)
ax.set_aspect('equal')
ax.set_title('whole image')


ax = fig.add_subplot(1,2,2)
da_sel = da.sel(pixel_x=slice(1750,2000)).sel(pixel_y=slice(850,1100))
da_sel.plot.imshow(ax=ax,robust=True,cmap = 'Greys', vmin=0, vmax=70)
ax.set_aspect('equal')
ax.set_title('around beam stop');

In [ ]:
# Same as above for detector artifacts

fig = plt.figure(figsize=(8,5))


ax = fig.add_subplot(1,1,1)
da_sel = da.sel(pixel_x=slice(200,400)).sel(pixel_y=slice(450,600))
da_sel.plot.imshow(ax=ax,robust=True,cmap = 'Greys')
ax.set_aspect('equal')
ax.set_title('showing detector bad pixels');

In [ ]:
# here we further zoom into defects and get line profile

fig = plt.figure(figsize=(12,5))


ax = fig.add_subplot(1,2,1)
da_sel = da.sel(pixel_x=slice(347-25,347+25)).sel(pixel_y=slice(552-25,552+25))
da_sel.plot.imshow(ax=ax,robust=True,cmap = 'Greys')
ax.axvline(x=347,color='r')
ax.set_aspect('equal')
ax.set_title('more zoom around bad pixels')

ax = fig.add_subplot(1,2,2)
da_sel.sel(pixel_x=347,method='nearest').plot()
ax.set_title('intensity profile along red line on the left\n(see intensity drop along the detector defect)');


> Notice that we are "slicing" the arrays in ways which allow us to zoom in on specific parts of the images.

Now, let's look into extracting an intensity profile from this Debye Scherre ring. There are other tools (which we will get to later) that do this for you, but here we can see the functionality of xarray.

In [ ]:
# This is similar way of selecting and averaging profile

fig = plt.figure(figsize=(10,10))

ax = fig.add_subplot(2,3,2)
da.plot.imshow(ax=ax,robust=True,cmap = 'Greys', vmin=0, vmax=70,add_colorbar=False)
ax.axhline(y=983,color='r',alpha=0.5)
ax.axvline(x=1900,color='b',alpha=0.5)
ax.set_aspect('equal')
ax.set_title('whole image')


ax = fig.add_subplot(2,2,3)
da_sel = da.sel(pixel_y=slice(983-5,983+5)).mean(dim='pixel_y') # here we first select and then average
da_sel.plot(ax=ax,color='r')
ax.set_title('intensity profile along red line above\n(+/-5 pixel averaged)')
ax.set_xlim([0,2048])

ax = fig.add_subplot(2,2,4)
da_sel = da.sel(pixel_x=slice(1900-5,1900+5)).mean(dim='pixel_x') # here we first select and then average
da_sel.plot(ax=ax,color='b')
ax.set_title('intensity profile along blue line above\n(+/-5 pixel averaged)')
ax.set_xlim([0,2048]);

We can even get an intensity profile along an arbitrary line.

In [ ]:
# this is more advanced way of getting line profile along yellow line below

fig = plt.figure(figsize=(10,5))

ax = fig.add_subplot(1,2,1)
da.plot.imshow(ax=ax,robust=True,cmap = 'Greys', vmin=0, vmax=70,add_colorbar=False)

# poi
p1 = (1900,983) # the center of the pattern
p2 = (10,2000)
# arrow
ax.arrow(p1[0],p1[1],p2[0]-p1[0],p2[1]-p1[1],width=20,color="y",length_includes_head=True)

ax.set_aspect('equal')
ax.set_title('whole image')



ax = fig.add_subplot(1,2,2)

from skimage.draw import line
# tuple storing x and y arrays representing corresponding x and y coordinates of our line
pline = line(p1[0],p1[1],p2[0],p2[1])
# creation of x_array from x coords of our line tuple
x = xr.DataArray(pline[0], dims="z")
# creation of x_array from y coords of our line tuple
y = xr.DataArray(pline[1], dims="z")
da_sel = da.interp(pixel_x=x,pixel_y=y)
da_sel.plot(ax=ax, color='b')

ax.set_title('intensity profile along yellow line');

In [ ]:
# We can also achieve this without using skimage, instead just
# using linear spaced lists to approximate the line. The skimage
# line isn't as "jagged" but it doesn't make much of a difference

fig = plt.figure(figsize=(10,5))

ax = fig.add_subplot(1,2,1)
da.plot.imshow(ax=ax,robust=True,cmap = 'Greys', vmin=0, vmax=70,add_colorbar=False)

# poi
p1 = (1900,983) # the center of the pattern
p2 = (10,2000)
# arrow
ax.arrow(p1[0],p1[1],p2[0]-p1[0],p2[1]-p1[1],width=20,color="y",length_includes_head=True)
ax.set_aspect('equal')
ax.set_title('whole image')




ax = fig.add_subplot(1,2,2)
da_int = da.interp(pixel_x=xr.DataArray(np.linspace(p1[0],p2[0],1000)),
                   pixel_y=xr.DataArray(np.linspace(p1[1],p2[1],1000)))
da_int.plot(ax=ax, color='y')

ax.set_title('intensity profile along yellow line');

Here is a simple azimuthal integrator written by <https://github.com/pmeshkov> & <https://github.com/KMorell>! Again, soon we will see tools for doing this efficiently, and with less "eyeballing".

In [ ]:
# adjustment of previously used center to get real center
# as well as fixed radii for blue rings
actual_center = np.add(p1,(17,-3))
radius = 315
tolerance = 10
radius1 = 200#radius-tolerance
radius2 = 1800#radius+tolerance


from skimage.draw import circle_perimeter

# finding every point in the sliver of the circle

circle = 0
data_values = da.values
sum_list = []
average_intensity = []

for i in np.arange(radius1, radius2):
    circle = circle_perimeter(actual_center[0],actual_center[1],i)
    for j in np.arange(0,len(circle[0])):
        try:
            sum_list = np.append(sum_list,data_values[circle[1][j]][circle[0][j]])
        except:
            continue
    average_intensity = np.append(average_intensity,np.average(sum_list))
    sum_list = []
print("complete")

In [ ]:
fig = plt.figure(figsize=(10,5))
# plot whole image
ax = fig.add_subplot(1,2,1)
da.plot.imshow(ax=ax,robust=True,cmap = 'Greys', vmin=0, vmax=70,add_colorbar=False)
ax.set_title('whole image')

circle1 = plt.Circle(actual_center, radius1, color='b', fill=False)
circle2 = plt.Circle(actual_center, radius2, color='b', fill=False)
ax.add_patch(circle1)
ax.add_patch(circle2)

intensity_xarray = xr.DataArray(average_intensity)

ax = fig.add_subplot(1,2,2)
intensity_xarray.plot(color='purple')
ax.set_xlabel("Radius from center")
ax.set_ylabel("Intensity")
ax.set_yscale('log')

ax.set_title('Average intensity around theta vs Radius');